# NBA_stat_analysis

Created by Ying Ma (3/16/2018)

Load pre-saved game data from each event, average metrics before and after and then compare pre vs. post as a group

In [454]:
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup
import time
from datetime import date, datetime
from selenium import webdriver
import copy
from dateutil import relativedelta
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp, levene, bartlett
from glob import glob
import os

In [639]:
pathname = 'C:/Users/mysel/Dropbox/datascience bootcamp/NBA/csv/'
filenames =  glob(pathname + '*.csv')


In [646]:
df.columns

Index(['Unnamed: 0', 'Name', 'Event', 'ED', 'Pre/Post', 'Rk', 'Date', 'Offset',
       'Tm', ' ', 'Opp', 'Win', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'ORtg', 'DRtg', 'OR_Team',
       'DR_Team'],
      dtype='object')

In [645]:
features = list(df)[-7:-2]
results = pd.DataFrame()
for feature in features:

    df_sum=pd.DataFrame(columns={'Name','Event','ED','Pre_M','Post_M','Pre_V','Post_V','p_value','z_score','Pre_n','Post_n'})
    
    for i in range(0,len(filenames)):
        os.chdir(pathname)
        df = pd.read_csv(filenames[i],encoding = 'ISO-8859-1')
       
        df.rename(columns={'\xa0.1':'Win'}, inplace=True)
        df['Win'] = [int(df['Win'][i].split()[1][1:-1]) for i in range(len(df['Win']))]
        df = df.drop(df[df['PTS']==0].index)
     
        df_sum = df_sum.append(df.iloc[0,1:4], ignore_index= True)
        df_sum['Pre_M'][i]= np.mean(df[feature][df['Pre/Post'] == 'Pre'][df[feature][df['Pre/Post'] == 'Pre']!=0])
        df_sum['Post_M'][i]= np.mean(df[feature][df['Pre/Post'] == 'Post'][df[feature][df['Pre/Post'] == 'Post']!=0])
        df_sum['Pre_V'][i]= np.std(df[feature][df['Pre/Post'] == 'Pre'][df[feature][df['Pre/Post'] == 'Pre']!=0])
        df_sum['Post_V'][i]=np.std(df[feature][df['Pre/Post'] == 'Post'][df[feature][df['Pre/Post'] == 'Post']!=0])
        df_sum['Pre_n'][i]= sum(df[feature][df['Pre/Post'] == 'Pre']!=0)
        df_sum['Post_n'][i]= sum(df[feature][df['Pre/Post'] == 'Post']!=0)    
        df_sum['z_score'][i],df_sum['p_value'][i] =ttest_ind(df[feature][df['Pre/Post'] == 'Pre'], df[feature][df['Pre/Post'] == 'Post']);
#        df_sum['z_score'][i],df_sum['p_value'][i] =levene(df[feature][df['Pre/Post'] == 'Pre'], df[feature][df['Pre/Post'] == 'Post']);

    
    #df_sum['']
    df_sm = df_sum[['Name','Event','ED','Pre_M','Post_M','Pre_V','Post_V','Pre_n','Post_n','p_value','z_score']]
    s=[ttest_rel(df_sum['Pre_M'], df_sum['Post_M'])[1],
       ttest_rel(df_sum['Pre_M'][df_sum['Event']=='Engage'], df_sum['Post_M'][df_sum['Event']=='Engage'])[1],
       ttest_rel(df_sum['Pre_M'][df_sum['Event']=='Baby'], df_sum['Post_M'][df_sum['Event']=='Baby'])[1],
       ttest_rel(df_sum['Pre_M'][df_sum['Event']=='Dead of Family'], df_sum['Post_M'][df_sum['Event']=='Dead of Family'])[1]
      ]

    s = pd.Series(data = s)
    results = results.append(s, ignore_index= True)

C:\Users\mysel\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in greater

C:\Users\mysel\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in less

C:\Users\mysel\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning:

invalid value encountered in less_equal



In [648]:
df_sum[['Name','Event','ED','Pre_M','Post_M','Pre_V','Post_V','p_value','z_score','Pre_n','Post_n']]

,Name,Event,ED,Pre_M,Post_M,Pre_V,Post_V,p_value,z_score,Pre_n,Post_n
0,Al Horford,Baby,2015-02-23,101.615,104.5,9.81787,15.2069,0.59134,-0.544497,13,12
1,Al Horford,Baby,2016-11-27,101.833,106.8,7.77639,12.1666,0.388903,-0.881803,6,15
2,Carmelo Anthony,Baby,2007-03-07,109.667,107.294,5.64702,11.4366,0.527256,0.640486,12,17
3,Carmelo Anthony,Engage,2004-12-25,105.909,111.75,9.63379,10.1335,0.191111,-1.35088,11,12
4,Charlie Villanueva,Baby,2015-11-18,101.2,105.5,9.36803,11.4419,0.374886,-0.907619,10,12
5,Dirk Nowitzki,Baby,2015-03-24,108.786,112.444,10.1923,9.82187,0.42462,-0.814284,14,9
6,Dirk Nowitzki,Engage,2012-02-14,97.0833,102.6,12.0862,12.1095,NaN,NaN,13,15
7,Draymond Green,Baby,2016-12-23,95.4667,102.571,14.1415,9.825,0.143665,-1.50605,15,14
8,Dwyane Wade,Engage,2013-12-21,100.545,108.5,10.4391,10.2689,0.110958,-1.67177,11,10
9,Isaiah Thomas,Baby,2012-01-29,114.833,113.077,10.8385,9.73069,0.685977,0.409479,12,13


In [649]:
results.columns = ['Over all', 'Engage', 'Baby','Death of Family']
results.index = features
#results = np.round(results,decimals = 3)
results

,Over all,Engage,Baby,Death of Family
PTS,0.494623,0.423201,0.737125,0.586313
GmSc,0.342227,0.251870,0.687770,0.599816
+/-,0.127357,0.216473,0.411823,0.226348
ORtg,0.389048,0.201647,0.274495,0.034879
DRtg,0.000189,0.000776,0.008341,0.963037


In [575]:
#filename = player_name +'_'+ player_event+'_'+ event_index['Date'][kk].strftime('%Y%m%d')+'.csv'
results.to_csv('C:/Users/mysel/Dropbox/datascience bootcamp/NBA/performance.csv')

In [653]:
    df_sum=pd.DataFrame(columns={'Name','Event','ED','Pre_M','Post_M','Pre_V','Post_V','p_value','z_score','Pre_n','Post_n'})
    
    for i in range(0,len(filenames)):
        os.chdir(pathname)
        df = pd.read_csv(filenames[i],encoding = 'ISO-8859-1')
        df = df.drop(df[df['PTS']==0].index)
        
        feature = 'DRtg'
        df_sum = df_sum.append(df.iloc[0,1:4], ignore_index= True)
        df_sum['Pre_M'][i]= np.mean(df[feature][df['Pre/Post'] == 'Pre'][df[feature][df['Pre/Post'] == 'Pre']!=0])
        df_sum['Post_M'][i]= np.mean(df[feature][df['Pre/Post'] == 'Post'][df[feature][df['Pre/Post'] == 'Post']!=0])
        df_sum['Pre_V'][i]= np.std(df[feature][df['Pre/Post'] == 'Pre'][df[feature][df['Pre/Post'] == 'Pre']!=0])
        df_sum['Post_V'][i]=np.std(df[feature][df['Pre/Post'] == 'Post'][df[feature][df['Pre/Post'] == 'Post']!=0])
        df_sum['Pre_n'][i]= sum(df[feature][df['Pre/Post'] == 'Pre']!=0)
        df_sum['Post_n'][i]= sum(df[feature][df['Pre/Post'] == 'Post']!=0)    
        df_sum['z_score'][i],df_sum['p_value'][i] =ttest_ind(df[feature][df['Pre/Post'] == 'Pre'], df[feature][df['Pre/Post'] == 'Post']);
#        df_sum['z_score'][i],df_sum['p_value'][i] =levene(df[feature][df['Pre/Post'] == 'Pre'], df[feature][df['Pre/Post'] == 'Post']);


C:\Users\mysel\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in greater

C:\Users\mysel\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in less

C:\Users\mysel\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning:

invalid value encountered in less_equal



## plotting results

In [657]:

import plotly.plotly as py 
import plotly.graph_objs as go

from plotly.graph_objs import *
py.sign_in('myself0116', '5KHVlvJQsZkxxLpkSk32')


trace0 = go.Scatter(
    x = np.linspace(np.floor(np.min(df_sum['Pre_M'])).astype(int),np.ceil(np.max(df_sum['Pre_M'])).astype(int)),
    y = np.linspace(np.floor(np.min(df_sum['Pre_M'])).astype(int),np.ceil(np.max(df_sum['Pre_M'])).astype(int)),
    mode = 'line', 
    name ='Pre = Post',
    opacity = 0.3,
    line = dict(color ='rgba(0, 0, 0, 1)', width = 2,dash = 'dash'),
)

trace1 = go.Scatter(
    x = df_sum['Pre_M'][df_sum['Event']=='Engage'],
    y = df_sum['Post_M'][df_sum['Event']=='Engage'],
    mode = 'markers+text', 
    text = df_sum['Name'][df_sum['Event']=='Engage'],
    textposition='bottom',
    name ='Engage',
    marker = dict(size = 10, color = 'rgba(255, 100, 100, .6)', line = dict(width = 2))
)

trace2 = go.Scatter(
    x = df_sum['Pre_M'][df_sum['Event']=='Baby'],
    y = df_sum['Post_M'][df_sum['Event']=='Baby'],    
    name ='New Born',
    text = df_sum['Name'][df_sum['Event']=='Baby'],
    textposition='bottom',
    mode = 'markers+text', 
    marker = dict(size = 10, color = 'rgba(100, 255, 100, .6)', line = dict(width = 2))
)

trace3 = go.Scatter(
    x = df_sum['Pre_M'][df_sum['Event']=='Dead of Family'],
    y = df_sum['Post_M'][df_sum['Event']=='Dead of Family'],
    mode = 'markers+text', 
    text = df_sum['Name'][df_sum['Event']=='Dead of Family'],
    textposition='bottom',
    name ='Death of Family',
    marker = dict(size = 10, color = 'rgba(100, 100, 255, .6)', line = dict(width = 2))
)

data = [trace1, trace2, trace3,trace0 ]
layout = Layout(title="Defense Rating",
                xaxis=dict(title='Pre Event', range = [92,119]),
                yaxis = dict(title='Post Event', range = [92,119]),
                height=650,
                width=700
                )
fig = Figure(data=data, layout=layout)

py.iplot(fig, filename ='Defense Rating')

## Correcting for team's influence

In [658]:
features = list(df)[-5:]
results = pd.DataFrame()
for feature in features:

    df_sum=pd.DataFrame(columns={'Name','Event','ED','OR_diff_pre','OR_diff_post','DR_diff_pre','DR_diff_post','Pre_n','Post_n'})
    
    for i in range(0,len(filenames)):
        os.chdir(pathname)
        df = pd.read_csv(filenames[i],encoding = 'ISO-8859-1')
        df['OR_diff'] = df['ORtg']- df['OR_Team']
        df['DR_diff'] = df['DRtg']- df['DR_Team']
       # feature = 'DRtg'
        df_sum = df_sum.append(df.iloc[0,1:4], ignore_index= True)
        df_sum['OR_diff_pre'][i]= np.mean(df['OR_diff'][df['Pre/Post'] == 'Pre'])
        df_sum['OR_diff_post'][i]= np.mean(df['OR_diff'][df['Pre/Post'] == 'Post'])
        df_sum['DR_diff_pre'][i]= np.mean(df['DR_diff'][df['Pre/Post'] == 'Pre'])
        df_sum['DR_diff_post'][i]= np.mean(df['DR_diff'][df['Pre/Post'] == 'Post'])
        
        df_sum['Pre_n'][i]= sum(df[feature][df['Pre/Post'] == 'Pre']!=0)
        df_sum['Post_n'][i]= sum(df[feature][df['Pre/Post'] == 'Post']!=0)    
       # df_sum['z_score'][i],df_sum['p_value'][i] =ttest_ind(df[feature][df['Pre/Post'] == 'Pre'], df[feature][df['Pre/Post'] == 'Post']);
#        df_sum['z_score'][i],df_sum['p_value'][i] =levene(df[feature][df['Pre/Post'] == 'Pre'], df[feature][df['Pre/Post'] == 'Post']);

    
    #df_sum['']
   # df_sm = df_sum[['Name','Event','ED','Pre_M','Post_M','Pre_V','Post_V','Pre_n','Post_n','p_value','z_score']]
    s=[ttest_rel(df_sum['OR_diff_pre'], df_sum['OR_diff_post'])[1],
       ttest_rel(df_sum['OR_diff_pre'][df_sum['Event']=='Engage'], df_sum['OR_diff_post'][df_sum['Event']=='Engage'])[1],
       ttest_rel(df_sum['OR_diff_pre'][df_sum['Event']=='Baby'], df_sum['OR_diff_post'][df_sum['Event']=='Baby'])[1],
       ttest_rel(df_sum['OR_diff_pre'][df_sum['Event']=='Dead of Family'], df_sum['OR_diff_post'][df_sum['Event']=='Dead of Family'])[1]
      ]
#    s=[ttest_rel(df_sum['Pre_V'], df_sum['Post_V'])[1],
#       ttest_rel(df_sum['Pre_V'][df_sum['Event']=='Engage'], df_sum['Post_V'][df_sum['Event']=='Engage'])[1],
#       ttest_rel(df_sum['Pre_V'][df_sum['Event']=='Baby'], df_sum['Post_V'][df_sum['Event']=='Baby'])[1],
#       ttest_rel(df_sum['Pre_V'][df_sum['Event']=='Dead of Family'], df_sum['Post_V'][df_sum['Event']=='Dead of Family'])[1]
#      ]
    s = pd.Series(data = s)
    results = results.append(s, ignore_index= True)

In [659]:
import plotly.plotly as py 
import plotly.graph_objs as go

from plotly.graph_objs import *
py.sign_in('myself0116', '5KHVlvJQsZkxxLpkSk32')


trace0 = go.Scatter(
    x = np.linspace(np.floor(np.min(df_sum['DR_diff_pre'])).astype(int),np.ceil(np.max(df_sum['DR_diff_pre'])).astype(int)),
    y = np.linspace(np.floor(np.min(df_sum['DR_diff_pre'])).astype(int),np.ceil(np.max(df_sum['DR_diff_pre'])).astype(int)),
    mode = 'line', 
    name ='Pre = Post',
    opacity = 0.3,
    line = dict(color ='rgba(0, 0, 0, 1)', width = 2,dash = 'dash'),
)

trace1 = go.Scatter(
    x = df_sum['DR_diff_pre'][df_sum['Event']=='Engage'],
    y = df_sum['DR_diff_post'][df_sum['Event']=='Engage'],
    mode = 'markers+text', 
    text = df_sum['Name'][df_sum['Event']=='Engage'],
    textposition='bottom',
    name ='Engage',
    marker = dict(size = 10, color = 'rgba(255, 100, 100, .6)', line = dict(width = 2))
)

trace2 = go.Scatter(
    x = df_sum['DR_diff_pre'][df_sum['Event']=='Baby'],
    y = df_sum['DR_diff_post'][df_sum['Event']=='Baby'],    
    name ='New Born',
    text = df_sum['Name'][df_sum['Event']=='Baby'],
    textposition='bottom',
    mode = 'markers+text', 
    marker = dict(size = 10, color = 'rgba(100, 255, 100, .6)', line = dict(width = 2))
)

trace3 = go.Scatter(
    x = df_sum['DR_diff_pre'][df_sum['Event']=='Dead of Family'],
    y = df_sum['DR_diff_post'][df_sum['Event']=='Dead of Family'],
    mode = 'markers+text', 
    text = df_sum['Name'][df_sum['Event']=='Dead of Family'],
    textposition='bottom',
    name ='Death of Family',
    marker = dict(size = 10, color = 'rgba(100, 100, 255, .6)', line = dict(width = 2))
)

data = [trace1, trace2, trace3,trace0 ]
layout = Layout(title="Defense Rating",
#                xaxis=dict(title='Pre Event', range = [-0.1,0.1]),
#                yaxis = dict(title='Post Event', range = [-0.1,0.1]),
                height=650,
                width=700
                )
fig = Figure(data=data, layout=layout)

py.iplot(fig, filename ='Defense Rating')